#**Two-sample Z-test on independent samples.**

#Case 1. Two individual measurements.
###The melting point of two specimens of material are measured to be $202\pm 3$ K and $209\pm 4$ K. Could these be two samples of different materials?
###(assumption: errors are Gaussian)

##$H_{_0}$: the samples are of the same substance. $H_{_1}$: the samples are made of different materials.
##If $H_{_0}$ is true, the melting points of the two samples should be identical within measurement errors. That is, $T_{_1} - T_{_2}$ should have a Gaussian distribution around mean $0$.

In [ ]:
from scipy.stats import norm
significance = 0.05
T1 = 202.0
T2 = 209.0
dT1 = 3.0
dT2 = 4.0
DeltaT = np.abs(T2 - T1) #only the magnitude of the difference matters
sigmaT = np.sqrt(dT1**2 + dT2**2)
DT_by_sigT = DeltaT / sigmaT
print("Standard deviation on difference: {} K".format(np.round(sigmaT, decimals = 3)))
print("Difference in melting points: {} K".format(np.round(DeltaT, decimals = 3)))
print("Difference in # of standard deviations: {}".format(np.round(DT_by_sigT, decimals = 3)))
#p-value (two-tailed) = P(|Z| > DT_by_sigT) = 2 * P(Z < -DT_by_sigT) = 2 * CDF(-DT_by_sigT)
pvalue = np.round(2 * norm.cdf(-DT_by_sigT), decimals = 3)
print("p-value corresponding to this difference: {}".format(pvalue))
if pvalue > significance:
  print("p-value > significance, cannot reject null hypothesis.")
else:
  print("p-value < significance, null hypothesis rejected!")

Standard deviation on difference: 5.0 K
Difference in melting points: 7.0 K
Difference in # of standard deviations: 1.4
p-value corresponding to this difference: 0.162
p-value > significance, cannot reject null hypothesis.


#Case 2. Two sets of measurements.
###The average score on the statistics exam in 2019 was 12.5 for 109 students. In 2020, it was 12.7 for 123 students. The distributions have a spread of $\sigma = 2.0$. Is the standard of students improving?

##$H_{_0}$: the standard has not changed. $H_{_1}$: the standard of students has increased (one-tailed test).
##If $H_{_0}$ is true, the difference in means of the two years should be normally distributed about 0.

In [ ]:
from scipy.stats import norm
significance = 0.05
mean2019 = 12.5
N2019 = 109
mean2020 = 12.7
N2020 = 123
sigma = 2.0 #this is the standard deviation on an INDIVIDUAL score
Delta_mean = mean2020 - mean2019 #the sign of the mean also matters
sigma2019 = sigma / np.sqrt(N2019) #This is the standard deviation on the MEAN score of 2019
sigma2020 = sigma / np.sqrt(N2020) #This is the standard deviation on the MEAN score of 2020
sigma_mean = np.sqrt(sigma2019**2 + sigma2020**2)
Dmean_by_sigmean = Delta_mean / sigma_mean
print("Standard deviation on difference in means: {}".format(np.round(sigma_mean, decimals = 3)))
print("Difference in means: {}".format(np.round(Delta_mean, decimals = 3)))
print("Difference in # of standard deviations: {}".format(np.round(Dmean_by_sigmean, decimals = 3)))
#p-value (one-tailed) = P(Z > Dmean_by_sigmean) = 1 - CDF(Dmean_by_sigmean)
pvalue = np.round(1 - norm.cdf(Dmean_by_sigmean), decimals = 3)
print("p-value corresponding to this difference: {}".format(pvalue))
if pvalue > significance:
  print("p-value > significance, cannot reject null hypothesis.")
else:
  print("p-value < significance, null hypothesis rejected!")

Standard deviation on difference in means: 0.263
Difference in means: 0.2
Difference in # of standard deviations: 0.76
p-value corresponding to this difference: 0.224
p-value > significance, cannot reject null hypothesis.


# Two-sample $t$-test on independent and dependent/paired/matched/correlated samples.

> ### Assumptions:<br> - Samples are drawn from a Gaussian distribution.<br> - Sample sizes approximately equal.<br> - Samples have the same variance (relaxed versions exist: Welch's $t$-test).

## Independent samples

### Generate two independent samples from a Normal distribution

In [ ]:
from scipy.stats import norm
import numpy as np
N = 50; N1 = 50; N2 = 50
np.random.seed(seed)
#Notice the means are different by design!
x1 = norm.rvs(loc = 0.0, size = N); x2 = norm.rvs(loc = 1.0, size = N)

### Compute the $t$-statistic for the difference in means
#### $t = \displaystyle{\overline{x_1} - \overline{x_2}\over \sqrt{\displaystyle{\hat{\sigma_1^2}\over N} + \displaystyle{\hat{\sigma_2^2}\over N}}}$

In [ ]:
se1 = x1.std(ddof = 1) / np.sqrt(N) #standard error on sample mean of sample 1
se2 = x2.std(ddof = 1) / np.sqrt(N) #standard error on sample mean of sample 2
sed = np.sqrt(se1**2 + se2**2) #standard error on pooled sample
tstat = (x1.mean() - x2.mean()) / sed
#Total number of degrees of freedom in pooled sample is total number of points - 2
#      because two population means are estimated from the samples.
df = (N1 -1) + (N2 - 1)
print("The t-statistic is {} and has {} degrees of freedom".format(np.round(tstat, decimals = 3),df))

The t-statistic is -6.72 and has 98 degrees of freedom


### Determine $p$-value of the observed $t$-statistic

In [ ]:
from scipy.stats import t
alpha = 0.05

choice = input("Perform one-tailed test? [Y]/N: ")
if choice.upper() == 'Y':
    print("OK, performing one-tailed test...")
    if tstat < 0:
        pvalue = t.cdf(tstat, df)
        print("Observed t statistic is negative, performing left-tailed test")
        print("p-value for t <= observed value is ", np.format_float_scientific(pvalue, precision=2))
    else:
        pvalue = 1 - t.cdf(tstat, df)
        print("Observed t statistic is positive, performing right-tailed test")
        print("p-value for t >= observed value is ", np.format_float_scientific(pvalue, precision=2))
else:
    tstat_abs = np.abs(tstat)
    pvalue = t.cdf(-tstat_abs, df) + 1 - t.cdf(tstat_abs, df)
    print("OK, performing two-tailed test...")
    print("p-value for |t| >= observed value is ", np.format_float_scientific(pvalue, precision=2))

if pvalue <= alpha:
    print("Since p-value is <= alpha, the null hypothesis is rejected.")
else:
    print("Since p-value is > alpha, the null hypothesis cannot be rejected.")

KeyboardInterrupt: ignored

### Try the same with `scipy.stats.ttest_ind` (two-tailed test)
#### p-value for one-tailed test should be half of that for a two-tailed test since the $t$ distribution is symmetric.

In [ ]:
from scipy.stats import ttest_ind
tstat, pvalue = ttest_ind(x1, x2, equal_var = True)
print("The observed t-statistic is", np.round(tstat, decimals = 3))
print("The corresponding p-value is ", np.format_float_scientific(pvalue, precision=2))


if pvalue <= alpha:
    print("Since p-value is <= alpha, the null hypothesis is rejected.")
else:
    print("Since p-value is > alpha, the null hypothesis cannot be rejected.")

The observed t-statistic is 2.31
The corresponding p-value is  2.3e-02
Since p-value is <= alpha, the null hypothesis is rejected.


## Dependent samples

### Generate two dependent samples

In [ ]:
np.random.seed(seed)
#x1: source + background with mean 0.5
#x2: background-subtracted data
x1 = norm.rvs(loc = 3.0, size = 50); x2 = x1 - norm.rvs(loc = 0.5, size = 50)
#Demonstrate that x1 and x2 are correlated
from scipy.stats import pearsonr
r, _ = pearsonr(x1, x2)
print("The two datasets have a Pearson r correlation coefficient of {}".format(np.round(r, decimals = 3)))

The two datasets have a Pearson r correlation coefficient of 0.68


### Compute the $t$-statistic for the difference in means
#### $t = \displaystyle{\overline{x_1}-\overline{x_2}\over \displaystyle{s_y\over \sqrt{N}}}$, where $s_y \equiv \sqrt{\displaystyle{1\over N-1}\Bigg[\displaystyle\sum\limits_{i=1}^N (x_{1,i}-x_{2,i})^2 - 
N(\overline{x_1}-\overline{x_2})^2
%\displaystyle{\Big(\displaystyle\sum\limits_{i=1}^N (x_{1,i}-x_{2,i})\Big)^2\over N}
\Bigg]}$

In [ ]:
from scipy.stats import ttest_rel
tstat, pvalue = ttest_rel(x1, x2)
print("The observed t-statistic is", np.round(tstat, decimals = 3))
print("The corresponding p-value is ", np.format_float_scientific(pvalue, precision=2))


if pvalue <= alpha:
    print("Since p-value is <= alpha, the null hypothesis is rejected.")
else:
    print("Since p-value is > alpha, the null hypothesis cannot be rejected.")

The observed t-statistic is 3.884
The corresponding p-value is  3.08e-04
Since p-value is <= alpha, the null hypothesis is rejected.
